In [1]:
import numpy as np
import math

#***(1) 讀入資料後，使用corpus=news.split(sep='\n') : 使用此方法將corpus斷開，取得每一份document。***
fp = open("TMBD_news_files_2000.txt","r")
news = fp.read()
corpus = news.split(sep='\n')

#***(2) 讀入stopword & punctuation並合併為Removewords。***
stopword = open("stopword_chinese.txt","r")
stopwords = stopword.read()
sws = stopwords.split()

punctuation = open("punctuation.txt","r")
punc = punctuation.read()
pc = punc.split()

sws.extend(pc)
Removewords = sws

#***(3) 定義function來移除Removewords。***
def removeStopwords(tokens,sws):
    return [w for w in tokens if w not in sws]

#***(4) 變為小寫並分割出各document的token。***
doctoken = []
for c in range(0,len(corpus)):
    doctoken.append(removeStopwords((corpus[c].lower().split()),Removewords))

#***(5) 移除第一個值（文章開頭之編號）。***
for i in range(0,len(doctoken)):
    del doctoken[i][0]
#print(doctoken[0][0])


In [2]:
#***(6) 得到corpus的term_dictionary。***
term_lib = news.lower().split()
term_dict = set(removeStopwords(term_lib,Removewords))
term_dict = list(term_dict)

In [ ]:
#***(7) 定義func來計算tf(use numpy to create array)。***
def termFrequency(doctoken):
    tf_vector = np.zeros([len(term_dict),1])
    for t in doctoken:
        tf_vector[term_dict.index(t),0] = doctoken.count(t)
    return tf_vector
tf_doc = []
for d in range(0,len(doctoken)):
    tf_doc.append(termFrequency(doctoken[d]))

In [ ]:
#***(8) 用numpy產生空的idf_vector，定義func計算idf
#***   (先求df再利用判斷式判斷df值，並藉此計算idf值，後塞入idf_vector)。***
idf_vector = np.zeros([len(term_dict),1])
def inverseDocumentFrequency(term):
    numDocumentsWithThisTerm = 0
    for doc in range(0,len(doctoken)):
            if term in doctoken[doc]:
                numDocumentsWithThisTerm = numDocumentsWithThisTerm + 1
    
    if numDocumentsWithThisTerm > 0:
        idf_vector[term_dict.index(term),0] = math.log10(float(len(doctoken)) / numDocumentsWithThisTerm)
    else:
        idf_vector[term_dict.index(term),0] = 0
    return idf_vector

for t in term_dict:
    inverseDocumentFrequency(t)

In [ ]:
#***(9) 求tfidf，迴圈對每篇document的tf值去乘上idf值。***
tfidf_doc = []
for doc in range(0,len(tf_doc)):
    tfidf_doc.append(tf_doc[doc]*idf_vector)

In [ ]:
#***(10) 定義func，一個計算tfidf值的平方和開根號，另一個計算相似度。***
def sqrt_sum_square(tfidf):
    return np.sqrt(sum(np.square(tfidf)))
    
def similarity(tfidf1,tfidf2): 
    return (np.dot(tfidf1.T,tfidf2)[0][0]) / ((sqrt_sum_square(tfidf1)[0]) * (sqrt_sum_square(tfidf2)[0]))

In [ ]:
#***(11) for迴圈計算各doc，與第56篇doc的相似度（因為起始值為0所以為55）。***
sim = []
for i in range(0,len(tfidf_doc)):
    sim.append(similarity(tfidf_doc[55],tfidf_doc[i]))

In [ ]:
#***(12) 給定空字典sim_dic，利用enumerate將各doc的相似度與doc名配對，並塞入sim_dic。***
sim_dic = dict()
for i,s in enumerate(sim):
    sim_dic['Doc%s' % (i+1)] = s

In [ ]:
#***(13) 按照value對sim_dic反向排序，並找出最大相似前10筆。***
sorted(sim_dic.values(),reverse=True)

In [ ]:
#***(14) 反向排序後，顯示排序後的key值。***
sorted(sim_dic, key=sim_dic.get,reverse=True)